In [1]:
from gurobi import *
import numpy as np
import pandas as pd
from scipy.spatial import distance
from itertools import chain, combinations

# PDSTSP


In [2]:
class Data:
    def __init__(self):
        self.customerNum = 0 
        self.nodeNum     = 0 
        self.droneNum    = 1
        self.cities      = []
        self.cor_X       = [] 
        self.cor_Y       = [] 
        self.serviceTime = [] 
        self.disMatrix   = [[]]
        self.dt          = None
        self.i_pot = None
        self.cus_can_served_by_drone = None
        self.drone_distances = None
        self.truck_distances = None
        
      
        

    def readData(self, path):
        self.dt = pd.read_csv(path, header = None).to_numpy()[:-1]
        self.customerNum = len(self.dt)
        self.i_pot = self.dt[0, 1:3]
        self.nodeNum = self.customerNum + 2 
        
        self.cities = [self.dt[i, 0] for i in range(len(self.dt))]
        
        self.cus_can_served_by_drone = [i for i in range(len(self.dt)) if self.dt[i, 3] == 0]
        
        self.drone_distances = [distance.euclidean((self.dt[i, 1:3]), self.i_pot)
                                if self.dt[i, 3] == 0 else float('inf')
                                for i in range(len(self.dt))]
        self.truck_distances = [[distance.cityblock(self.dt[i, 1:3], self.dt[j, 1:3])
                                 for i in range(len(self.dt))] for j in range(len(self.dt))]
    
    

        
       


In [3]:
data = Data()

data.readData("20140813T111857.csv")
print(data.dt)

print(data.customerNum)

[[ 0.   0.   0.   0.4]
 [ 1.   2.1  2.4  0. ]
 [ 2.   3.8  0.6  1. ]
 [ 3.   0.3  6.6  0. ]
 [ 4.   0.4  3.2  0. ]
 [ 5.   0.6  3.1  0. ]
 [ 6.   2.3 11.9  0. ]
 [ 7.   0.3  9.9  0. ]
 [ 8.   0.8  2.5  0. ]
 [ 9.   3.6 10.7  0. ]]
10


In [4]:
print(data.dt[1, 1:3])

[2.1 2.4]


In [5]:
print(data.cities)

[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, 9.0]


In [6]:
print(data.truck_distances)

[[0.0, 4.5, 4.3999999999999995, 6.8999999999999995, 3.6, 3.7, 14.2, 10.200000000000001, 3.3, 14.299999999999999], [4.5, 0.0, 3.4999999999999996, 5.999999999999999, 2.5000000000000004, 2.2, 9.7, 9.3, 1.4000000000000001, 9.799999999999999], [4.3999999999999995, 3.4999999999999996, 0.0, 9.5, 6.0, 5.699999999999999, 12.8, 12.8, 4.9, 10.299999999999999], [6.8999999999999995, 5.999999999999999, 9.5, 0.0, 3.4999999999999996, 3.7999999999999994, 7.300000000000001, 3.3000000000000007, 4.6, 7.4], [3.6, 2.5000000000000004, 6.0, 3.4999999999999996, 0.0, 0.30000000000000004, 10.6, 6.8, 1.1, 10.7], [3.7, 2.2, 5.699999999999999, 3.7999999999999994, 0.30000000000000004, 0.0, 10.5, 7.1000000000000005, 0.8000000000000002, 10.6], [14.2, 9.7, 12.8, 7.300000000000001, 10.6, 10.5, 0.0, 4.0, 10.9, 2.5000000000000013], [10.200000000000001, 9.3, 12.8, 3.3000000000000007, 6.8, 7.1000000000000005, 4.0, 0.0, 7.9, 4.1], [3.3, 1.4000000000000001, 4.9, 4.6, 1.1, 0.8000000000000002, 10.9, 7.9, 0.0, 11.0], [14.2999999

In [7]:
print(data.drone_distances)

[inf, 3.189043743820395, inf, 6.606814663663572, 3.22490309931942, 3.157530680769389, 12.120231020900551, 9.904544411531507, 2.6248809496813377, 11.289375536317321]


In [8]:
model = Model("PDSTSP")

Academic license - for non-commercial use only - expires 2021-07-04
Using license file /home/quanghuy205/gurobi.lic


In [9]:
#SET
N = [i for i in range (1, data.customerNum)]
G = [0] + N
M = [m for m in range(data.droneNum)]

N_d = data.cus_can_served_by_drone
N_t = [i for i in N if i not in N_d]
A = [(i,j) for i in G for j in G if i != j]
C_truck = {(i,j): data.truck_distances[i][j] for i,j in A}
C_drone = data.drone_distances

#Decision variables
# z_i = 0: if cus i is visited by vehicle, = 1 if visited by drones
# x_ij if (i->j) in vehicle tour
# # y_im = 1 if cus i assigned to drone m ()
z = [i for i in G]
x = [[[] for i in G] for j in G]  
y = [[[] for i in G] for m in M] 



In [10]:
        
#completion time
T = model.addVar(0, GRB.INFINITY, 1.0, GRB.CONTINUOUS, "traveltime" )
expr = LinExpr(0)
expr.addTerms(1.0, T)
model.setObjective(expr, GRB.MINIMIZE)
expr.clear()

In [11]:
#2
for i in G:
    for j in G:
        if i != j:
            x[i][j] = model.addVar(0.0, 1.0, 0.0, GRB.BINARY, name="x%d,%d" % (i, j))
            x[j][i] = x[i][j]
            expr = LinExpr(0)
            expr.addTerms(data.truck_distances[i][j], x[i][j])
        else:
            x[i][i] = model.addVar(0.0, 0.0, 0.0, GRB.BINARY, name="x%d,%d" % (i, i))

model.addConstr(T >= expr, "truckTime")
model.update()
expr.clear()
model.getVars()

[<gurobi.Var traveltime>,
 <gurobi.Var x0,0>,
 <gurobi.Var x0,1>,
 <gurobi.Var x0,2>,
 <gurobi.Var x0,3>,
 <gurobi.Var x0,4>,
 <gurobi.Var x0,5>,
 <gurobi.Var x0,6>,
 <gurobi.Var x0,7>,
 <gurobi.Var x0,8>,
 <gurobi.Var x0,9>,
 <gurobi.Var x1,0>,
 <gurobi.Var x1,1>,
 <gurobi.Var x1,2>,
 <gurobi.Var x1,3>,
 <gurobi.Var x1,4>,
 <gurobi.Var x1,5>,
 <gurobi.Var x1,6>,
 <gurobi.Var x1,7>,
 <gurobi.Var x1,8>,
 <gurobi.Var x1,9>,
 <gurobi.Var x2,0>,
 <gurobi.Var x2,1>,
 <gurobi.Var x2,2>,
 <gurobi.Var x2,3>,
 <gurobi.Var x2,4>,
 <gurobi.Var x2,5>,
 <gurobi.Var x2,6>,
 <gurobi.Var x2,7>,
 <gurobi.Var x2,8>,
 <gurobi.Var x2,9>,
 <gurobi.Var x3,0>,
 <gurobi.Var x3,1>,
 <gurobi.Var x3,2>,
 <gurobi.Var x3,3>,
 <gurobi.Var x3,4>,
 <gurobi.Var x3,5>,
 <gurobi.Var x3,6>,
 <gurobi.Var x3,7>,
 <gurobi.Var x3,8>,
 <gurobi.Var x3,9>,
 <gurobi.Var x4,0>,
 <gurobi.Var x4,1>,
 <gurobi.Var x4,2>,
 <gurobi.Var x4,3>,
 <gurobi.Var x4,4>,
 <gurobi.Var x4,5>,
 <gurobi.Var x4,6>,
 <gurobi.Var x4,7>,
 <gurobi.Var x

In [12]:
#3
for m in M:
    expr = LinExpr(0)
    for i in N:
        if i in N_d:
            y[m][i] = model.addVar(0.0, 1.0, 0.0, GRB.BINARY, name="y%d,%d" % (m, i) )
            expr.addTerms(data.drone_distances[i], y[m][i])
       
    model.addConstr(T >= expr, "dronetime")
    expr.clear()

model.update()


In [13]:
#4
for i in G:
        z[i] = model.addVar(0.0, 1.0, 0.0, GRB.BINARY, name = "z%d" % (i))


for i in N_t:
    expr = LinExpr(0)
    expr.addTerms(1.0, z[i])
    model.addConstr(expr == 1)
    expr.clear()


In [14]:
#4

for i in G:
    expr1 = LinExpr(0)
    expr2 = LinExpr(0)
    for j in range(i, data.customerNum):
        expr1.addTerms(1.0, x[i][j])
        
    expr2.addTerms(1.0, z[i])
    
    model.addConstr(expr1 == expr2)
    expr1.clear()
    expr2.clear()

In [15]:


for i in N_d:
    expr1 = LinExpr(0)
    expr2 = LinExpr(0)
    expr2.addTerms(1.0, z[i])
    
    for m in M:
        expr1.addTerms(1.0, y[m][i])
        
    model.addConstr(expr1 == 1 - expr2)
    expr1.clear()
    expr2.clear()

        
    

In [16]:

for i in N:
    expr1 = LinExpr(0) 
    expr2 = LinExpr(0) 
    
    for k in G:
        
            expr1.addTerms(1, x[k][i]) 
            print(k,i)
    for j in G:
        
            print(i,j)
            expr2.addTerms(1, x[i][j]) 
               
    model.addConstr(expr1 == expr2) 
    expr1.clear() 
    expr2.clear() 

0 1
1 1
2 1
3 1
4 1
5 1
6 1
7 1
8 1
9 1
1 0
1 1
1 2
1 3
1 4
1 5
1 6
1 7
1 8
1 9
0 2
1 2
2 2
3 2
4 2
5 2
6 2
7 2
8 2
9 2
2 0
2 1
2 2
2 3
2 4
2 5
2 6
2 7
2 8
2 9
0 3
1 3
2 3
3 3
4 3
5 3
6 3
7 3
8 3
9 3
3 0
3 1
3 2
3 3
3 4
3 5
3 6
3 7
3 8
3 9
0 4
1 4
2 4
3 4
4 4
5 4
6 4
7 4
8 4
9 4
4 0
4 1
4 2
4 3
4 4
4 5
4 6
4 7
4 8
4 9
0 5
1 5
2 5
3 5
4 5
5 5
6 5
7 5
8 5
9 5
5 0
5 1
5 2
5 3
5 4
5 5
5 6
5 7
5 8
5 9
0 6
1 6
2 6
3 6
4 6
5 6
6 6
7 6
8 6
9 6
6 0
6 1
6 2
6 3
6 4
6 5
6 6
6 7
6 8
6 9
0 7
1 7
2 7
3 7
4 7
5 7
6 7
7 7
8 7
9 7
7 0
7 1
7 2
7 3
7 4
7 5
7 6
7 7
7 8
7 9
0 8
1 8
2 8
3 8
4 8
5 8
6 8
7 8
8 8
9 8
8 0
8 1
8 2
8 3
8 4
8 5
8 6
8 7
8 8
8 9
0 9
1 9
2 9
3 9
4 9
5 9
6 9
7 9
8 9
9 9
9 0
9 1
9 2
9 3
9 4
9 5
9 6
9 7
9 8
9 9


In [17]:
expr = LinExpr(0)
for j in range(1, data.customerNum):
    expr.addTerms(1.0, x[0][j])
    
model.addConstr(expr <= 1)
expr.clear()        
            

In [18]:
#get all subtours
def powerset(iterable):
    "powerset([1,2,3]) --> () (1,) (2,) (3,) (1,2) (1,3) (2,3) (1,2,3)"
    s = list(iterable)
    return chain.from_iterable(combinations(s, r) for r in range(len(s)+1))

S = list(powerset(range(len(data.cities))))
# The first element of the list is the empty set and the last element is the full set, hence we remove them.
S = S[1:(len(S))]

print(len(S))
import sys
print(sys.getsizeof(S)/1024/1024," GB")
print(S)

1023
0.0078582763671875  GB
[(0,), (1,), (2,), (3,), (4,), (5,), (6,), (7,), (8,), (9,), (0, 1), (0, 2), (0, 3), (0, 4), (0, 5), (0, 6), (0, 7), (0, 8), (0, 9), (1, 2), (1, 3), (1, 4), (1, 5), (1, 6), (1, 7), (1, 8), (1, 9), (2, 3), (2, 4), (2, 5), (2, 6), (2, 7), (2, 8), (2, 9), (3, 4), (3, 5), (3, 6), (3, 7), (3, 8), (3, 9), (4, 5), (4, 6), (4, 7), (4, 8), (4, 9), (5, 6), (5, 7), (5, 8), (5, 9), (6, 7), (6, 8), (6, 9), (7, 8), (7, 9), (8, 9), (0, 1, 2), (0, 1, 3), (0, 1, 4), (0, 1, 5), (0, 1, 6), (0, 1, 7), (0, 1, 8), (0, 1, 9), (0, 2, 3), (0, 2, 4), (0, 2, 5), (0, 2, 6), (0, 2, 7), (0, 2, 8), (0, 2, 9), (0, 3, 4), (0, 3, 5), (0, 3, 6), (0, 3, 7), (0, 3, 8), (0, 3, 9), (0, 4, 5), (0, 4, 6), (0, 4, 7), (0, 4, 8), (0, 4, 9), (0, 5, 6), (0, 5, 7), (0, 5, 8), (0, 5, 9), (0, 6, 7), (0, 6, 8), (0, 6, 9), (0, 7, 8), (0, 7, 9), (0, 8, 9), (1, 2, 3), (1, 2, 4), (1, 2, 5), (1, 2, 6), (1, 2, 7), (1, 2, 8), (1, 2, 9), (1, 3, 4), (1, 3, 5), (1, 3, 6), (1, 3, 7), (1, 3, 8), (1, 3, 9), (1, 4, 5), (

In [19]:
for s in S:
    model.addConstr(quicksum(x[i][j] for i in s for j in G if j not in s) >= quicksum(z[i] for i in s) + 1 - len(s))
model.update()

In [20]:
model.optimize()

Gurobi Optimizer version 9.1.2 build v9.1.2rc0 (linux64)
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads
Optimize a model with 1054 rows, 119 columns and 28262 nonzeros
Model fingerprint: 0xa69cb24a
Variable types: 1 continuous, 118 integer (118 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+01]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 9e+00]
Found heuristic solution: objective 52.1173241
Presolve removed 1054 rows and 119 columns
Presolve time: 0.00s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.01 seconds
Thread count was 1 (of 12 available processors)

Solution count 2: 11.2894 52.1173 

Optimal solution found (tolerance 1.00e-04)
Best objective 1.128937553632e+01, best bound 1.128937553632e+01, gap 0.0000%


In [21]:
model.printAttr('X')



    Variable            X 
-------------------------
  traveltime      11.2894 
        x1,0            1 
        x3,2            1 
        x4,3            1 
        x6,5            1 
        x7,1            1 
        x7,4            1 
        x9,6            1 
        x9,7            1 
        x9,8            1 
        y0,9            1 
          z0            1 
          z1            1 
          z2            1 
          z3            1 
          z4            1 
          z5            1 
          z6            1 
          z7            1 
          z8            1 


In [22]:
model.getVars()

[<gurobi.Var traveltime (value 11.289375536317321)>,
 <gurobi.Var x0,0 (value 0.0)>,
 <gurobi.Var x0,1 (value 0.0)>,
 <gurobi.Var x0,2 (value 0.0)>,
 <gurobi.Var x0,3 (value 0.0)>,
 <gurobi.Var x0,4 (value 0.0)>,
 <gurobi.Var x0,5 (value 0.0)>,
 <gurobi.Var x0,6 (value 0.0)>,
 <gurobi.Var x0,7 (value 0.0)>,
 <gurobi.Var x0,8 (value 0.0)>,
 <gurobi.Var x0,9 (value 0.0)>,
 <gurobi.Var x1,0 (value 1.0)>,
 <gurobi.Var x1,1 (value 0.0)>,
 <gurobi.Var x1,2 (value 0.0)>,
 <gurobi.Var x1,3 (value 0.0)>,
 <gurobi.Var x1,4 (value 0.0)>,
 <gurobi.Var x1,5 (value 0.0)>,
 <gurobi.Var x1,6 (value 0.0)>,
 <gurobi.Var x1,7 (value 0.0)>,
 <gurobi.Var x1,8 (value 0.0)>,
 <gurobi.Var x1,9 (value 0.0)>,
 <gurobi.Var x2,0 (value 0.0)>,
 <gurobi.Var x2,1 (value 0.0)>,
 <gurobi.Var x2,2 (value 0.0)>,
 <gurobi.Var x2,3 (value 0.0)>,
 <gurobi.Var x2,4 (value 0.0)>,
 <gurobi.Var x2,5 (value 0.0)>,
 <gurobi.Var x2,6 (value 0.0)>,
 <gurobi.Var x2,7 (value 0.0)>,
 <gurobi.Var x2,8 (value 0.0)>,
 <gurobi.Var x2,9 (

In [23]:
data.drone_distances[1] + data.drone_distances[4] + data.drone_distances[5] + data.drone_distances[8] + data.drone_distances[9] 

23.485734009907862

In [24]:
data.truck_distaces[0,1] + 

SyntaxError: invalid syntax (<ipython-input-24-33163a41114f>, line 1)